Comparison between exterpiment 1 and 2, with enphasys over how the emotion enhancement changes the results.

In [13]:
# imports
import numpy as np
import csv
import json

In [14]:
# File path

EXP_1_CSV_FILE_PATH = "/work/cvcs2024/SLR_sentiment_enhanced/SLRSE_model_data/Ensemble/SL-GCN/prediction/1/stat.csv"
EXP_2_CSV_FILE_PATH = "/work/cvcs2024/SLR_sentiment_enhanced/SLRSE_model_data/Ensemble/SL-GCN/prediction/2/stat.csv"

EMOTION_LABEL_PATH = "/homes/mcolombari/SLR_Sentiment_Enhanced/DAN_module/sign_emotion_assosiation.json"

In [15]:
# Load emotion label

f_w_to_e = open(EMOTION_LABEL_PATH)
w_to_e = json.load(f_w_to_e)

raw_label = []
for _, i in w_to_e.items():
    for a in i:
        raw_label.append(a)

In [16]:
# Load data csv

exp_1_dict = dict()
with open(EXP_1_CSV_FILE_PATH, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        exp_1_dict[str(row[0])] = {
            'label': int(row[1]),
            'score_diff': float(row[2]),
            'score_diff_softmax': float(row[3]),
            'score': np.array([float(row[i + 4]) for i in range(2000)])
        }


exp_2_dict = dict()
with open(EXP_2_CSV_FILE_PATH, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        exp_2_dict[str(row[0])] = {
            'label': int(row[1]),
            'score_diff': float(row[2]),
            'score_diff_softmax': float(row[3]),
            'emotion': np.array([float(row[i + 4]) for i in range(8)]),
            'score': np.array([float(row[i + 12]) for i in range(2000)])
        }

In [17]:
# Print first element dataset
print(f"Exp 1 dataset: {exp_1_dict['signer0_sample4']}")
print(f"Exp 2 dataset: {exp_2_dict['signer0_sample4']}")

Exp 1 dataset: {'label': 38, 'score_diff': 717.9242, 'score_diff_softmax': 1.996, 'score': array([-2.63238406,  0.58455741, -5.64533567, ..., -0.14928038,
        2.12425494, -5.85246658])}
Exp 2 dataset: {'label': 38, 'score_diff': 717.9242, 'score_diff_softmax': 1.996, 'emotion': array([0.3403, 0.    , 0.    , 0.    , 0.    , 0.2825, 0.    , 0.3772]), 'score': array([-2.63238406,  0.58455741, -5.64533567, ..., -0.14928038,
        2.12425494, -5.85246658])}


## Analysis First Experiment

We are working with the provided ensemble with just the alpha values changed accordisngly to the accuracy of our model.
We introduce 2 new losses, one where we normalize the score vector and compute the sum of the module difference with respect to the optimal vector (1 on lable, 0 everywere else), and the second where we do the same difference but we apply the softmax as normalization.


In [18]:
top1_count = sum([1 if el['label'] == el['score'].argsort()[-1] else 0 for _ , el in exp_1_dict.items()])
print(f"Top1 accuracy: {top1_count / len(exp_1_dict)}")

top5_count = sum([1 if el['label'] in el['score'].argsort()[-5:] else 0 for _ , el in exp_1_dict.items()])
print(f"Top5 accuracy: {top5_count / len(exp_1_dict)}")

top10_count = sum([1 if el['label'] in el['score'].argsort()[-10:] else 0 for _ , el in exp_1_dict.items()])
print(f"Top10 accuracy: {top10_count / len(exp_1_dict)}")

Top1 accuracy: 0.2824091076019097
Top5 accuracy: 0.599706206390011
Top10 accuracy: 0.6999632757987514


## Analysis 2 experiment

Now we enhance with emeotion, we take the emotion vector multiply by EMOTION_SLOPE and sum by EMOTION_OFFSET and the use it to increment the score in percentage:
    
    ret += ret * ((e_weight * EMOTION_SLOPE) + EMOTION_OFFSET)

In [19]:
top1_count = sum([1 if el['label'] == el['score'].argsort()[-1] else 0 for _ , el in exp_2_dict.items()])
print(f"Top1 accuracy: {top1_count / len(exp_2_dict)}")

top5_count = sum([1 if el['label'] in el['score'].argsort()[-5:] else 0 for _ , el in exp_2_dict.items()])
print(f"Top5 accuracy: {top5_count / len(exp_2_dict)}")

top10_count = sum([1 if el['label'] in el['score'].argsort()[-10:] else 0 for _ , el in exp_2_dict.items()])
print(f"Top10 accuracy: {top10_count / len(exp_2_dict)}")

Top1 accuracy: 0.28094013955196473
Top5 accuracy: 0.6000734484024972
Top10 accuracy: 0.6999632757987514


In [20]:
# Accuracy for emotion label

em_value = {}
for k, el in exp_2_dict.items():
    if el['label'] in raw_label:
        em_value[k] = el


top1_count = sum([1 if el['label'] == el['score'].argsort()[-1] else 0 for _ , el in em_value.items()])
print(f"Top1 accuracy: {top1_count / len(em_value)}")

top5_count = sum([1 if el['label'] in el['score'].argsort()[-5:] else 0 for _ , el in em_value.items()])
print(f"Top5 accuracy: {top5_count / len(em_value)}")

top10_count = sum([1 if el['label'] in el['score'].argsort()[-10:] else 0 for _ , el in em_value.items()])
print(f"Top10 accuracy: {top10_count / len(em_value)}")

Top1 accuracy: 0.45098039215686275
Top5 accuracy: 0.7843137254901961
Top10 accuracy: 0.7843137254901961


In [21]:
# compute table of result with respect to emotion enhance.
# TP: video that should be enhance and are, in the wight way.
# TN: video that shouldn't  be enhance and are not, in any way.
# FP: video that shouldn't  be enhance and are.
# FN: video that should  be enhance and are not.

file_name_to_enahnce = [k for k, el in exp_2_dict.items() if el['label'] in raw_label]
file_name_not_to_enahnce = [k for k, el in exp_2_dict.items() if not el['label'] in raw_label]

print(f"Total file: {len(exp_2_dict)}")
print(f"Total file to enhance: {len(file_name_to_enahnce)}")
print(f"Total file NOT to enhance: {len(file_name_not_to_enahnce)}")

#   TP - FN
tp = 0
fn = 0
for name in file_name_to_enahnce:
    label = exp_2_dict[name]['label']
    # Find the corresponding emotion
    emotion_to_enhance = next(int(key) for key, value in w_to_e.items() if label in value)
    emotion = exp_2_dict[name]['emotion']

    # I consider any kind of enhancemnt as such, so it's TP just by a "0.001".
    if emotion[emotion_to_enhance] > 0:
        tp += 1
    else:
        fn += 1


#   TN - FP
tn = 0
fp = 0
for name in file_name_not_to_enahnce:
    emotion = exp_2_dict[name]['emotion']
    if emotion[1:].sum() == 0:
        tn += 1
    else:
        fp += 1


print(f"\nTP: {tp}")
print(f"TN: {tn}")
print(f"FP: {fp}")
print(f"FN: {fn}")
print(f"\nTotal: {tp + tn + fp + fn}")

Total file: 2723
Total file to enhance: 51
Total file NOT to enhance: 2672

TP: 32
TN: 376
FP: 2296
FN: 19

Total: 2723


In [35]:
# What if we enhance only the top10 prediction?

file_name_to_enahnce = [k for k, el in exp_2_dict.items() if el['label'] in raw_label]
file_name_not_to_enahnce = [k for k, el in exp_2_dict.items() if not el['label'] in raw_label]

print(f"Total file: {len(exp_2_dict)}")
print(f"Total file to enhance: {len(file_name_to_enahnce)}")
print(f"Total file NOT to enhance: {len(file_name_not_to_enahnce)}")

#   TP - FN
tp = 0
fn = 0
for name in file_name_to_enahnce:
    label = exp_2_dict[name]['label']
    # Find the corresponding emotion
    emotion_to_enhance = next(int(key) for key, value in w_to_e.items() if label in value)
    emotion = exp_2_dict[name]['emotion']

    prev_score = exp_1_dict[name]['score'].argsort()[-10:]
    top10_to_enhance = [p for p in prev_score if p in w_to_e[str(emotion_to_enhance)]]

    # print(prev_score)
    # print(w_to_e[str(emotion_to_enhance)])

    # I consider any kind of enhancemnt as such, so it's TP just by a "0.001".
    if len(top10_to_enhance) > 0 and emotion[emotion_to_enhance] > 0:
        tp += 1
    else:
        fn += 1


#   TN - FP
tn = 0
fp = 0
for name in file_name_not_to_enahnce:
    emotion = exp_2_dict[name]['emotion']
    prev_score = exp_1_dict[name]['score'].argsort()[-10:]

    union = [p for p in prev_score if p in raw_label] # label that would be enhanced.

    # emotion correspondign to the label enhaced.
    emotion_enhanced = [int(k) for k, v in w_to_e.items() if len([u for u in union if u in v])>0]

    if len(union) == 0 or emotion[emotion_enhanced].sum() == 0:
        tn += 1
    else:
        fp += 1


print(f"\nTP: {tp}")
print(f"TN: {tn}")
print(f"FP: {fp}")
print(f"FN: {fn}")
print(f"\nTotal: {tp + tn + fp + fn}")

Total file: 2723
Total file to enhance: 51
Total file NOT to enhance: 2672

TP: 26
TN: 2574
FP: 98
FN: 25

Total: 2723


that's great by enhancing the top10 we reduce drastically the FP and increase the TN.